In [1]:
from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *
import plotly.express as px

from mainnet_launch.constants import *

destinations_df = get_full_table_as_df(Destinations, where_clause=Destinations.chain_id == 1)
blocks_df = get_full_table_as_df(Blocks, where_clause=Blocks.chain_id == 1)
event_df = get_full_table_as_df(RebalanceEvents)
plan_df = get_full_table_as_df(RebalancePlans)
trasactions_df = get_full_table_as_df(Transactions)
destination_tokens_df = get_full_table_as_df(DestinationTokens)


rebalance_df = merge_tables_as_df(
    selectors=[
        TableSelector(
            RebalanceEvents,
        ),
        TableSelector(Transactions, [Transactions.block], join_on=(Transactions.tx_hash == RebalanceEvents.tx_hash)),
        TableSelector(
            Blocks,
            [Blocks.datetime],
            (Transactions.block == Blocks.block) & (Transactions.chain_id == Blocks.chain_id),
        ),
    ],
    where_clause=(RebalanceEvents.autopool_vault_address == AUTO_USD.autopool_eth_addr),
    order_by=Blocks.datetime,
)

tokens_df = merge_tables_as_df(
    [
        TableSelector(
            DestinationTokens,
        ),
        TableSelector(
            table=Tokens,
            select_fields=[Tokens.symbol, Tokens.decimals],
            join_on=(DestinationTokens.token_address == Tokens.token_address),
        ),
    ],
)

destination_to_underlying = destinations_df.set_index("destination_vault_address")["underlying_symbol"].to_dict()

rebalance_df["destination_in_symbol"] = rebalance_df["destination_in"].map(destination_to_underlying)
rebalance_df["destination_out_symbol"] = rebalance_df["destination_out"].map(destination_to_underlying)
rebalance_df["move_name"] = rebalance_df["destination_out_symbol"] + " -> " + rebalance_df["destination_in_symbol"]


destination_token_address_to_symbols = (
    tokens_df.groupby("destination_vault_address")["symbol"].apply(tuple).apply(str).to_dict()
)
rebalance_df["destination_in_tokens"] = rebalance_df["destination_in"].map(destination_token_address_to_symbols)
rebalance_df["destination_out_tokens"] = rebalance_df["destination_out"].map(destination_token_address_to_symbols)
rebalance_df["tokens_move_name"] = (
    rebalance_df["destination_out_tokens"] + " -> " + rebalance_df["destination_in_tokens"]
)

# take out a 100 - 90 swap cost = 10
rebalance_df["spot_swap_cost"] = rebalance_df["spot_value_out"] - rebalance_df["spot_value_in"]
rebalance_df["spot_slippage_bps"] = 10_000 * rebalance_df["spot_swap_cost"] / rebalance_df["spot_value_out"]

rebalance_df["safe_swap_cost"] = rebalance_df["safe_value_out"] - rebalance_df["safe_value_in"]
rebalance_df["safe_slippage_bps"] = 10_000 * rebalance_df["safe_swap_cost"] / rebalance_df["safe_value_out"]

2025-05-30 16:59:16.305 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


2025-05-30 16:59:17,845 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-05-30 16:59:17,848 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-30 16:59:18,007 INFO sqlalchemy.engine.Engine select current_schema()
2025-05-30 16:59:18,009 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-30 16:59:18,163 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-05-30 16:59:18,166 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-05-30 16:59:18,326 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-30 16:59:18,327 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

2025-05-30 16:59:21.875 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 16:59:21.958 
  command:

    streamlit run /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-30 16:59:21.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 16:59:21.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 16:59:21.959 No runtime found, using MemoryCacheStorageManager


2025-05-30 16:59:22,035 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-30 16:59:22,036 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-30 16:59:22,036 INFO sqlalchemy.engine.Engine [cached since 3.71s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x12bbcd1e0>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-30 16:59:22,036 INFO sqlalchemy.engine.Engine SELECT
    rebalance_events.*,
    transactions.block,
    blocks.datetime
FROM rebalance_events
JOIN transactions
  ON transact

2025-05-30 16:59:22.461 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 16:59:22.461 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-05-30 16:59:22,714 INFO sqlalchemy.engine.Engine ROLLBACK


2025-05-30 16:59:22.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 16:59:22.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 16:59:22.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 16:59:22.794 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 16:59:22.795 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-05-30 16:59:22,877 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-30 16:59:22,877 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-05-30 16:59:22,878 INFO sqlalchemy.engine.Engine [cached since 4.551s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x12bbcd6f0>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-05-30 16:59:22,878 INFO sqlalchemy.engine.Engine SELECT
    destination_tokens.*,
    tokens.symbol,
    tokens.decimals
FROM destination_tokens
JOIN tokens
  ON destination_to

2025-05-30 16:59:23.115 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-30 16:59:23.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
rebalance_df[rebalance_df["tx_hash"] == "0xa48c0d19f6e7b4a41c7557a957a6b748075491d5eba08112bf010708b43da7bf"].T

,403
tx_hash,0xa48c0d19f6e7b4a41c7557a957a6b748075491d5eba0...
autopool_vault_address,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
chain_id,1
rebalance_file_path,rebalance_plan_1748632467_0xa7569A44f348d3D70d...
destination_out,0xb949C40F18fB52747d96850696c954718168fde3
destination_in,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee
quantity_out,297218.628078
quantity_in,289200.547509
safe_value_out,300000.891656
safe_value_in,299783.841545


In [3]:
plan_df[plan_df["autopool_vault_address"] == AUTO_USD.autopool_eth_addr].sort_values("datetime_generated").tail(3).T

,4467,4468,4466
file_name,rebalance_plan_1748646398_0xa7569A44f348d3D70d...,rebalance_plan_1748646569_0xa7569A44f348d3D70d...,rebalance_plan_1748648141_0xa7569A44f348d3D70d...
datetime_generated,2025-05-30 23:06:38+00:00,2025-05-30 23:09:29+00:00,2025-05-30 23:35:41+00:00
autopool_vault_address,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
chain_id,1,1,1
solver_address,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9
rebalance_type,Pool2Pool,Pool2Pool,Pool2Pool
destination_out,0xb949C40F18fB52747d96850696c954718168fde3,0xb949C40F18fB52747d96850696c954718168fde3,0xb949C40F18fB52747d96850696c954718168fde3
token_out,0x81A2612F6dEA269a6Dd1F6DeAb45C5424EE2c4b7,0x81A2612F6dEA269a6Dd1F6DeAb45C5424EE2c4b7,0x81A2612F6dEA269a6Dd1F6DeAb45C5424EE2c4b7
destination_in,0x9b18801C9E95f45AD0aDD54ef2Eb4690c44d170d,0x9b18801C9E95f45AD0aDD54ef2Eb4690c44d170d,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee
token_in,0xcf62F905562626CfcDD2261162a51fd02Fc9c5b6,0xcf62F905562626CfcDD2261162a51fd02Fc9c5b6,0xfD1627E3f3469C8392C8c3A261D8F0677586e5e1


In [ ]:
from mainnet_launch.data_fetching.get_state_by_block import *

# map decimals → normalizer
dec_norm = {
    6: safe_normalize_6_with_bool_success,
    18: safe_normalize_with_bool_success,
}

autoUSD_flash_borrow_solver = "0xD02b50CFc6c2903bF13638B28D081ad11515B6f9"

balance_of_calls = []
for _, row in tokens_df.iterrows():
    token = row["token_address"]
    normalizer = dec_norm[row["decimals"]]
    call = Call(
        token,
        ["balanceOf(address)(uint256)", autoUSD_flash_borrow_solver],
        [(token, normalizer)],
    )
    balance_of_calls.append(call)

before = get_state_by_one_block(balance_of_calls, 22597581 - 1, ETH_CHAIN)
after = get_state_by_one_block(balance_of_calls, 22597581, ETH_CHAIN)

In [5]:
diff = {}
for k in before.keys():
    if after[k] is not None:
        diff[k] = after[k] - before[k]
diff
# looks like there was an extra $22 frxUSD left in the pool

{'0x83F20F44975D03b1b09e64809B757c47f942BEeA': 0.0,
 '0xbf5495Efe5DB9ce00f80364C8B423567e58d2110': 0.0,
 '0xae78736Cd615f374D3085123A210448E74Fc6393': 0.0,
 '0x0655977FEb2f289A4aB78af67BAB0d17aAb84367': 0.0,
 '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E': 0.0,
 '0x853d955aCEf822Db058eb8505911ED77F175b99e': 0.0,
 '0x856c4Efb76C1D1AE02e20CEB03A2A6a08b0b8dC3': 0.0,
 '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2': 0.0,
 '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0': 0.0,
 '0x9D39A5DE30e57443BfF2A8307A4256c8797A3497': 0.0,
 '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704': 0.0,
 '0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD': 0.0,
 '0xA35b1B31Ce002FBF2058D22F30f95D405200A15b': 0.0,
 '0xD4fa2D31b7968E448877f69A96DE69f5de8cD23E': 0.0,
 '0xdAC17F958D2ee523a2206206994597C13D831ec7': 0.0,
 '0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6': 0.0,
 '0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee': 0.0,
 '0x5E8422345238F34275888049021821E8E08CAa1f': 0.0,
 '0x7Bc3485026Ac48b6cf9BaF0A377477Fff5703Af8': 0.0,
 '0xA0b86991

In [6]:
new_tx_hash = "0x3b5ab1bad915b3b34caaece1ebe33c4d86220ee4412e5d9c0503ed6a85c358f1"
new_block = 22597905

In [7]:
rebalance_df[rebalance_df["tx_hash"] == new_tx_hash].T

,404
tx_hash,0x3b5ab1bad915b3b34caaece1ebe33c4d86220ee4412e...
autopool_vault_address,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
chain_id,1
rebalance_file_path,rebalance_plan_1748636368_0xa7569A44f348d3D70d...
destination_out,0xb949C40F18fB52747d96850696c954718168fde3
destination_in,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee
quantity_out,198173.042719
quantity_in,192828.437946
safe_value_out,200000.396346
safe_value_in,199888.465544


In [8]:
plan_df[plan_df["autopool_vault_address"] == AUTO_USD.autopool_eth_addr].sort_values("datetime_generated").tail(1).T

,4466
file_name,rebalance_plan_1748648141_0xa7569A44f348d3D70d...
datetime_generated,2025-05-30 23:35:41+00:00
autopool_vault_address,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
chain_id,1
solver_address,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9
rebalance_type,Pool2Pool
destination_out,0xb949C40F18fB52747d96850696c954718168fde3
token_out,0x81A2612F6dEA269a6Dd1F6DeAb45C5424EE2c4b7
destination_in,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee
token_in,0xfD1627E3f3469C8392C8c3A261D8F0677586e5e1


In [9]:
before = get_state_by_one_block(balance_of_calls, new_block - 1, ETH_CHAIN)
after = get_state_by_one_block(balance_of_calls, new_block, ETH_CHAIN)
diff = {}
for k in before.keys():
    if after[k] is not None:
        diff[k] = after[k] - before[k]
diff
# looks like th

{'0x83F20F44975D03b1b09e64809B757c47f942BEeA': 0.0,
 '0xbf5495Efe5DB9ce00f80364C8B423567e58d2110': 0.0,
 '0xae78736Cd615f374D3085123A210448E74Fc6393': 0.0,
 '0x0655977FEb2f289A4aB78af67BAB0d17aAb84367': 0.0,
 '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E': 0.0,
 '0x853d955aCEf822Db058eb8505911ED77F175b99e': 0.0,
 '0x856c4Efb76C1D1AE02e20CEB03A2A6a08b0b8dC3': 0.0,
 '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2': 0.0,
 '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0': 0.0,
 '0x9D39A5DE30e57443BfF2A8307A4256c8797A3497': 0.0,
 '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704': 0.0,
 '0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD': 0.0,
 '0xA35b1B31Ce002FBF2058D22F30f95D405200A15b': 0.0,
 '0xD4fa2D31b7968E448877f69A96DE69f5de8cD23E': 0.0,
 '0xdAC17F958D2ee523a2206206994597C13D831ec7': 0.0,
 '0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6': 0.0,
 '0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee': 0.0,
 '0x5E8422345238F34275888049021821E8E08CAa1f': 0.0,
 '0x7Bc3485026Ac48b6cf9BaF0A377477Fff5703Af8': 0.0,
 '0xA0b86991

In [10]:
10000 * (118.48162424869952 - 15) / 200_000

5.1740812124349755

In [11]:
118 - 15

103

In [12]:
rebalance_df[rebalance_df["tx_hash"] == "0x2403575c107f3bc9d59b77466f360f79c0e7405127a1200a96c72bff84947eaf"].T

,405
tx_hash,0x2403575c107f3bc9d59b77466f360f79c0e7405127a1...
autopool_vault_address,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
chain_id,1
rebalance_file_path,rebalance_plan_1748641689_0xa7569A44f348d3D70d...
destination_out,0xb949C40F18fB52747d96850696c954718168fde3
destination_in,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee
quantity_out,148619.473709
quantity_in,144605.576806
safe_value_out,150000.594478
safe_value_in,149902.62369


In [13]:
10_000 * ((105 - 11) / 150_000)

6.266666666666667

In [14]:
before = get_state_by_one_block(balance_of_calls, 22598346 - 1, ETH_CHAIN)
after = get_state_by_one_block(balance_of_calls, 22598346, ETH_CHAIN)
diff = {}
for k in before.keys():
    if after[k] is not None:
        diff[k] = after[k] - before[k]
diff

{'0x83F20F44975D03b1b09e64809B757c47f942BEeA': 0.0,
 '0xbf5495Efe5DB9ce00f80364C8B423567e58d2110': 0.0,
 '0xae78736Cd615f374D3085123A210448E74Fc6393': 0.0,
 '0x0655977FEb2f289A4aB78af67BAB0d17aAb84367': 0.0,
 '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E': 0.0,
 '0x853d955aCEf822Db058eb8505911ED77F175b99e': 0.0,
 '0x856c4Efb76C1D1AE02e20CEB03A2A6a08b0b8dC3': 0.0,
 '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2': 0.0,
 '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0': 0.0,
 '0x9D39A5DE30e57443BfF2A8307A4256c8797A3497': 0.0,
 '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704': 0.0,
 '0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD': 0.0,
 '0xA35b1B31Ce002FBF2058D22F30f95D405200A15b': 0.0,
 '0xD4fa2D31b7968E448877f69A96DE69f5de8cD23E': 0.0,
 '0xdAC17F958D2ee523a2206206994597C13D831ec7': 0.0,
 '0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6': 0.0,
 '0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee': 0.0,
 '0x5E8422345238F34275888049021821E8E08CAa1f': 0.0,
 '0x7Bc3485026Ac48b6cf9BaF0A377477Fff5703Af8': 0.0,
 '0xA0b86991

In [15]:
for k, v in before.items():
    if v:
        if v > 0:
            print(k, v)

0x9D39A5DE30e57443BfF2A8307A4256c8797A3497 8.064052714370002
0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48 19.782192
0x4c9EDD5852cd905f086C759E8383e09bff1E68B3 82.5927227344909
0xCAcd6fd266aF91b8AeD52aCCc382b4e165586E29 69.49839527906306


In [16]:
for k, v in after.items():
    if v:
        if v > 0:
            print(k, v)

0x9D39A5DE30e57443BfF2A8307A4256c8797A3497 8.064052714370002
0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48 19.782192
0x4c9EDD5852cd905f086C759E8383e09bff1E68B3 82.5927227344909
0xCAcd6fd266aF91b8AeD52aCCc382b4e165586E29 80.90912117767435


In [17]:
before["0xCAcd6fd266aF91b8AeD52aCCc382b4e165586E29"]

69.49839527906306

In [18]:
after["0xCAcd6fd266aF91b8AeD52aCCc382b4e165586E29"]

80.90912117767435

In [19]:
rebalance_df

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,...,destination_in_symbol,destination_out_symbol,move_name,destination_in_tokens,destination_out_tokens,tokens_move_name,spot_swap_cost,spot_slippage_bps,safe_swap_cost,safe_slippage_bps
0,0x67618958fa10c91e18218c3c3f7eb57509e70445e898...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,85.899838,100.008658,100.118752,100.008658,...,autoUSD,sUSDe,sUSDe -> autoUSD,"('USDC',)","('sUSDe',)","('sUSDe',) -> ('USDC',)",0.058898,5.885779,0.110094,10.996327
1,0x5513250273bfdbf939553c9ed98ce48b8479bc739634...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,0x6F628dcCD275feA4277722D177265741031E09d7,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,98.326351,99.954654,99.992786,99.954654,...,autoUSD,crvUSDUSDT-f,crvUSDUSDT-f -> autoUSD,"('USDC',)","('crvUSD', 'USDT')","('crvUSD', 'USDT') -> ('USDC',)",0.025546,2.555152,0.038132,3.813499
2,0x14425c16bdc198d43d738991130a7fea735c85d7c883...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,25503.391475,25348.990383,25503.391475,25505.925983,...,crv2pool,autoUSD,autoUSD -> crv2pool,"('USDT', 'USDC')","('USDC',)","('USDC',) -> ('USDT', 'USDC')",0.735512,0.288398,-2.534508,-0.993792
3,0x5c3a705003f40a1e38309a79c873f54f147043dbcdd4...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x7583b1589aDD33320366A48A92794D77763FAE9e,19127.543600,18806.816895,19127.543600,19124.181930,...,crvUSDUSDT-f,autoUSD,autoUSD -> crvUSDUSDT-f,"('USDT', 'crvUSD')","('USDC',)","('USDC',) -> ('USDT', 'crvUSD')",5.091897,2.662076,3.361670,1.757502
4,0x8ac67f7b06d5dcaa032ce596da03410177deaac9028a...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1743792976_0xa7569A44f348d3D70d...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,14345.657699,14261.656811,14345.657699,14349.979251,...,crv2pool,autoUSD,autoUSD -> crv2pool,"('USDT', 'USDC')","('USDC',)","('USDC',) -> ('USDT', 'USDC')",-1.212511,-0.845211,-4.321552,-3.012446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,0xa5211a055efbf9c16c315a2b84f111502ccf3a5338cc...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1748625848_0xa7569A44f348d3D70d...,0xb949C40F18fB52747d96850696c954718168fde3,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,74317.247448,72318.271916,75000.297269,74964.035894,...,SkySaver,frxsUSDS,frxsUSDS -> SkySaver,"('scrvUSD', 'sUSDS')","('frxUSD', 'sUSDS')","('frxUSD', 'sUSDS') -> ('scrvUSD', 'sUSDS')",37.414470,4.988586,36.261375,4.834831
403,0xa48c0d19f6e7b4a41c7557a957a6b748075491d5eba0...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1748632467_0xa7569A44f348d3D70d...,0xb949C40F18fB52747d96850696c954718168fde3,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,297218.628078,289200.547509,300000.891656,299783.841545,...,SkySaver,frxsUSDS,frxsUSDS -> SkySaver,"('scrvUSD', 'sUSDS')","('frxUSD', 'sUSDS')","('frxUSD', 'sUSDS') -> ('scrvUSD', 'sUSDS')",218.448005,7.281622,217.050111,7.234982
404,0x3b5ab1bad915b3b34caaece1ebe33c4d86220ee4412e...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,rebalance_plan_1748636368_0xa7569A44f348d3D70d...,0xb949C40F18fB52747d96850696c954718168fde3,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,198173.042719,192828.437946,200000.396346,199888.465544,...,SkySaver,frxsUSDS,frxsUSDS -> SkySaver,"('scrvUSD', 'sUSDS')","('frxUSD', 'sUSDS')","('frxUSD', 'sUSDS') -> ('scrvUSD', 'sUSDS')",118.481624,5.924075,111.930802,5.596529
405,0x2403575c107f3bc9d59b77466f360f79c0e7405127a1...,0xa7569A44f348d3D70

In [20]:
px.scatter(rebalance_df, x="spot_slippage_bps", y="safe_slippage_bps")

In [21]:
px.histogram(rebalance_df["safe_slippage_bps"] - rebalance_df["spot_slippage_bps"])

In [22]:
rebalance_df.iloc[-4:].T

,403,404,405,406
tx_hash,0xa48c0d19f6e7b4a41c7557a957a6b748075491d5eba0...,0x3b5ab1bad915b3b34caaece1ebe33c4d86220ee4412e...,0x2403575c107f3bc9d59b77466f360f79c0e7405127a1...,0x9110705aa3f081a6eafb0a2bdd50174cf53ea10ac593...
autopool_vault_address,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
chain_id,1,1,1,1
rebalance_file_path,rebalance_plan_1748632467_0xa7569A44f348d3D70d...,rebalance_plan_1748636368_0xa7569A44f348d3D70d...,rebalance_plan_1748641689_0xa7569A44f348d3D70d...,rebalance_plan_1748648141_0xa7569A44f348d3D70d...
destination_out,0xb949C40F18fB52747d96850696c954718168fde3,0xb949C40F18fB52747d96850696c954718168fde3,0xb949C40F18fB52747d96850696c954718168fde3,0xb949C40F18fB52747d96850696c954718168fde3
destination_in,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee
quantity_out,297218.628078,198173.042719,148619.473709,148616.67598
quantity_in,289200.547509,192828.437946,144605.576806,144609.424124
safe_value_out,300000.891656,200000.396346,150000.594478,149997.77075
safe_value_in,299783.841545,199888.465544,149902.62369,149944.21039


In [23]:
10000 * (218 - 22) / 300_000

6.533333333333333

In [24]:
10000 * (118 - 15) / 200_000

5.15

In [25]:
10000 * (105 - 11) / 150_000

6.266666666666667

In [26]:
most_recent = "0x9110705aa3f081a6eafb0a2bdd50174cf53ea10ac59365b3dcc07cbe709a5999"

In [27]:
rebalance_df.iloc[-4:].T

,403,404,405,406
tx_hash,0xa48c0d19f6e7b4a41c7557a957a6b748075491d5eba0...,0x3b5ab1bad915b3b34caaece1ebe33c4d86220ee4412e...,0x2403575c107f3bc9d59b77466f360f79c0e7405127a1...,0x9110705aa3f081a6eafb0a2bdd50174cf53ea10ac593...
autopool_vault_address,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
chain_id,1,1,1,1
rebalance_file_path,rebalance_plan_1748632467_0xa7569A44f348d3D70d...,rebalance_plan_1748636368_0xa7569A44f348d3D70d...,rebalance_plan_1748641689_0xa7569A44f348d3D70d...,rebalance_plan_1748648141_0xa7569A44f348d3D70d...
destination_out,0xb949C40F18fB52747d96850696c954718168fde3,0xb949C40F18fB52747d96850696c954718168fde3,0xb949C40F18fB52747d96850696c954718168fde3,0xb949C40F18fB52747d96850696c954718168fde3
destination_in,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee,0x3Ee3F7A0B244004B0678EBa98bA091eC101513Ee
quantity_out,297218.628078,198173.042719,148619.473709,148616.67598
quantity_in,289200.547509,192828.437946,144605.576806,144609.424124
safe_value_out,300000.891656,200000.396346,150000.594478,149997.77075
safe_value_in,299783.841545,199888.465544,149902.62369,149944.21039


In [32]:
45 / 150000

0.0003

In [31]:
before = get_state_by_one_block(balance_of_calls, 22598878 - 1, ETH_CHAIN)
after = get_state_by_one_block(balance_of_calls, 22598878, ETH_CHAIN)
diff = {}
for k in before.keys():
    if after[k] is not None:
        diff[k] = after[k] - before[k]
diff

{'0x83F20F44975D03b1b09e64809B757c47f942BEeA': 0.0,
 '0xbf5495Efe5DB9ce00f80364C8B423567e58d2110': 0.0,
 '0xae78736Cd615f374D3085123A210448E74Fc6393': 0.0,
 '0x0655977FEb2f289A4aB78af67BAB0d17aAb84367': 0.0,
 '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E': 0.0,
 '0x853d955aCEf822Db058eb8505911ED77F175b99e': 0.0,
 '0x856c4Efb76C1D1AE02e20CEB03A2A6a08b0b8dC3': 0.0,
 '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2': 0.0,
 '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0': 0.0,
 '0x9D39A5DE30e57443BfF2A8307A4256c8797A3497': 0.0,
 '0xBe9895146f7AF43049ca1c1AE358B0541Ea49704': 0.0,
 '0xa3931d71877C0E7a3148CB7Eb4463524FEc27fbD': 0.0,
 '0xA35b1B31Ce002FBF2058D22F30f95D405200A15b': 0.0,
 '0xD4fa2D31b7968E448877f69A96DE69f5de8cD23E': 0.0,
 '0xdAC17F958D2ee523a2206206994597C13D831ec7': 0.0,
 '0x04C154b66CB340F3Ae24111CC767e0184Ed00Cc6': 0.0,
 '0xCd5fE23C85820F7B72D0926FC9b05b43E359b7ee': 0.0,
 '0x5E8422345238F34275888049021821E8E08CAa1f': 0.0,
 '0x7Bc3485026Ac48b6cf9BaF0A377477Fff5703Af8': 0.0,
 '0xA0b86991